In [ ]:
변호사들이 질문하는 패턴은 좀 다릅니다.
완전.. 키워드 패턴입니다.

진짜 해보면 RAG 마다 "의미" "키워드" 기반 검색 비율이 너무 다르더라고요

변호사들의 질문 형태는 의미기반(semantic) 접근이 일반적입니다. 특히 복잡한 법적 문제를 다룰 때, 질문은 특정 법률 조항이나 판례의 맥락을 파악하고 적용하는 방식으로 이루어집니다. 이런 경우, 질문은 단순한 키워드 이상의 의미를 내포하며, 법적 해석이나 논리를 이해해야 합니다.

예를 들어, 한 법적 상황에서 “이 사건에서 계약 불이행이 성립하나요?”라는 질문은 키워드로만 접근하기 어렵습니다. 의미기반 접근은 해당 상황의 법률적 의미를 파악하고, 유사한 판례나 법률 조항을 고려해야 답할 수 있습니다.

다만, 간단한 법률 검색이나 문서 검토 등에서는 키워드 기반 검색도 유용하게 사용됩니다.

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

1. BM25은 키워드 검색이라서 질문 역시 키워드 형태로 분리를 해야 한다.
2. FAISS는 전통적인 TD-IDF이라서 질문을 그대로 입력해서 검색한다. ** 앙상블은 이게 안된다.. ㅠ,.ㅜ

그래서 하나의 질문으로 위 2가지 조건을 만족할수 없을거 같아서. 아래와 같이 각각 검색을 시켰더니... 
한번해보셔요. 놀라실겁니다. 
주요 변경사항은 다음과 같습니다:

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc["content"] for doc in docs)

# Kiwi 토크나이저를 추가했습니다:
from kiwipiepy import Kiwi

# load_resources 함수에 Kiwi 인스턴스를 추가했습니다:
kiwi = Kiwi()

# 쿼리를 토큰화하는 새로운 함수를 추가했습니다:
def tokenize_query(query: str, kiwi: Kiwi) -> str:
    tokens = kiwi.tokenize(query)
    return " ".join([token.form for token in tokens])

# process_retriever_output 함수를 수정하여 FAISS와 BM25에 대해 다른 쿼리 처리 방식을 적용했습니다:
def process_retriever_output(question: str, faiss_retriever, bm25_retriever, kiwi: Kiwi):
    faiss_docs = faiss_retriever.get_relevant_documents(question)
    tokenized_question = tokenize_query(question, kiwi)
    bm25_docs = bm25_retriever.get_relevant_documents(tokenized_question)
    
    # Combine and deduplicate documents 
    all_docs = faiss_docs + bm25_docs
    unique_docs = list({doc.page_content: doc for doc in all_docs}.values())
    
    context = format_docs(unique_docs)
    #sources = get_sources(unique_docs)
    return {"context": context, "sources": sources, "question": question}

# 메인 함수와 체인 구성을 수정하여 새로운 검색 방식을 반영했습니다.

In [ ]:


from kiwipiepy import Kiwi
# Initialize Kiwi
kiwi = Kiwi()

def tokenize_query(query: str) -> str:
    tokens = kiwi.tokenize(query)
    return " ".join([token.form for token in tokens])

def combined_search(query: str, faiss_retriever, bm25_retriever):
    faiss_docs = faiss_retriever.get_relevant_documents(query)
    tokenized_query = tokenize_query(query)
    bm25_docs = bm25_retriever.get_relevant_documents(tokenized_query)
    
    # Combine and deduplicate documents
    all_docs = faiss_docs + bm25_docs
    unique_docs = list({doc.page_content: doc for doc in all_docs}.values())
    
    return unique_docs

In [ ]:
EnsembleRetriever는 두 개의 검색기(FAISS와 BM25)의 결과를 결합하여 최종적으로 가장 관련성이 높은 문서를 반환하는 방법입니다. 각 검색기의 결과에 가중치를 부여하여, BM25의 결과가 더 큰 영향을 미치도록 설계되었습니다. 이 방식은 다양한 검색 알고리즘의 장점을 활용하여 더 나은 검색 결과를 제공합니다.

[상세한 답변]
EnsembleRetriever는 두 개의 서로 다른 검색 알고리즘인 FAISS와 BM25를 사용하여 각각 10개의 문서를 검색합니다. 이 두 검색기의 결과는 다음과 같은 방식으로 결합됩니다:

검색기 결과: FAISS와 BM25는 각각 k=10으로 설정되어 10개의 문서를 검색합니다.
가중치 적용: FAISS의 결과에는 0.3의 가중치가, BM25의 결과에는 0.7의 가중치가 적용됩니다. 이는 BM25의 결과가 최종 순위에 더 큰 영향을 미치도록 하기 위함입니다.
최종 점수 계산: 각 문서에 대해 최종 점수는 다음과 같이 계산됩니다:
최종 점수 = (FAISS 점수 * 0.3) + (BM25 점수 * 0.7)
상위 문서 반환: 최종 점수가 가장 높은 상위 10개의 문서가 반환됩니다.

예를 들어, 문서 A에 대해 FAISS가 0.9점을 주고 BM25가 0.5점을 주었다면:
최종 점수 = (0.9 * 0.3) + (0.5 * 0.7) = 0.27 + 0.35 = 0.62

반면, 문서 B에 대해 FAISS가 0.6점을 주고 BM25가 0.8점을 주었다면:
최종 점수 = (0.6 * 0.3) + (0.8 * 0.7) = 0.18 + 0.56 = 0.74

이 경우, 문서 B가 더 높은 최종 점수를 받게 되어 상위 결과에 포함될 가능성이 높습니다.


In [ ]:

# EnsembleRetriever의 예시 코드
from typing import List, Dict

def calculate_final_score(faiss_score: float, bm25_score: float) -> float:
    """FAISS와 BM25 점수를 기반으로 최종 점수를 계산하는 함수입니다."""
    faiss_weight = 0.3  # FAISS의 가중치
    bm25_weight = 0.7   # BM25의 가중치
    final_score = (faiss_score * faiss_weight) + (bm25_score * bm25_weight)
    return final_score

# 예시 문서 점수
document_a_scores = (0.9, 0.5)  # FAISS 점수, BM25 점수
document_b_scores = (0.6, 0.8)  # FAISS 점수, BM25 점수

# 최종 점수 계산
final_score_a = calculate_final_score(*document_a_scores)
final_score_b = calculate_final_score(*document_b_scores)

print(f"문서 A의 최종 점수: {final_score_a}")  # 문서 A의 최종 점수 출력
print(f"문서 B의 최종 점수: {final_score_b}")  # 문서 B의 최종 점수 출력

In [ ]:
혹시나 해서 .앙상블 수치의 작동원리 입니다. 다들 아시겠지만.  단순 순서만 재조정 합니다. 
두 retriever (FAISS와 BM25)는 각각 10개의 문서를 검색합니다 (k=10).
ensemble_retriever는 각 retriever의 결과를 결합합니다.
FAISS의 결과에는 0.3의 가중치가, BM25의 결과에는 0.7의 가중치가 적용됩니다.
최종적으로 가중 점수가 가장 높은 상위 10개의 문서가 반환됩니다.

이 과정을 통해:

BM25의 결과가 최종 순위에 더 큰 영향을 미칩니다 (0.7 > 0.3).
하지만 FAISS가 매우 높은 점수를 준 문서도 여전히 상위 결과에 포함될 수 있습니다.

예를 들어, FAISS가 문서 A에 0.9점을 주고 BM25가 0.5점을 준 경우:

최종 점수 = (0.9 * 0.3) + (0.5 * 0.7) = 0.27 + 0.35 = 0.62

반면, FAISS가 0.6점, BM25가 0.8점을 준 문서 B의 경우:

최종 점수 = (0.6 * 0.3) + (0.8 * 0.7) = 0.18 + 0.56 = 0.74

이 경우, 문서 B가 더 높은 최종 점수를 받게 됩니다.
여러가지 서비스를 구현하다가 내가 20년넘게 해온 업무를 직접 며칠간 래깅해봤지만. 원하는 수준이 안되서 .
왜 그러나 해서 파봤습니다.
그냥 포장지에 있는 메뉴얼대로  높을 불에 10분, 뚜껑열고 3분 다음에 드시면 된다는게 아니라.
맛을 보고서야.. 아.. ... 내가 알고 있던 라면맛이 이거였는데..
한번 시도해보셔요. 
벡터 검색이랑, 키워드 검색이랑 .